# 🔮 Python Dunder Methods Masterclass

## 📚 Table of Contents
1. Introduction to Dunder Methods
2. Object Lifecycle Dunders
3. Representation Dunders
4. Attribute Access Dunders
5. Numeric Operation Dunders
6. Container Dunders
7. Callable and Context Manager Dunders
8. Descriptor Dunders
9. Advanced Applications
10. Real-World Examples

## 🎯 Learning Objectives
After completing this notebook, you will:
- Master all common dunder methods
- Understand object lifecycle management
- Implement custom container types
- Create descriptors and properties
- Build advanced Python classes

## 1. Introduction to Dunder Methods 🔮

```
Common Dunder Categories:
┌─────────────────────────┐
│ Lifecycle               │ __new__, __init__, __del__
│ Representation          │ __str__, __repr__, __format__
│ Attribute Access        │ __getattr__, __setattr__, __delattr__
│ Descriptors            │ __get__, __set__, __delete__
│ Container Methods      │ __len__, __getitem__, __setitem__
│ Numeric Operations     │ __add__, __sub__, __mul__
│ Comparison            │ __eq__, __lt__, __gt__
│ Context Management    │ __enter__, __exit__
└─────────────────────────┘
```

## 2. Object Lifecycle Dunders 🔄

In [ ]:
class LifecycleDemo:
    def __new__(cls, *args, **kwargs):
        print(f"1. __new__: Creating new {cls.__name__} instance")
        instance = super().__new__(cls)
        return instance
    
    def __init__(self, name: str):
        print(f"2. __init__: Initializing {self.__class__.__name__}")
        self.name = name
    
    def __del__(self):
        print(f"3. __del__: Cleaning up {self.__class__.__name__}")

# Singleton Pattern using __new__
class Singleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        # Initialize only once
        if not hasattr(self, 'initialized'):
            self.initialized = True
            self.data = {}

# Example usage
obj = LifecycleDemo("test")
del obj

# Singleton example
s1 = Singleton()
s2 = Singleton()
print(f"Same instance: {s1 is s2}")

## 3. Advanced Attribute Access 🔑

In [ ]:
class AttributeTracker:
    def __init__(self):
        self._data = {}
        self._history = []
    
    def __getattr__(self, name: str) -> Any:
        print(f"Accessing {name}")
        if name not in self._data:
            raise AttributeError(f"'{self.__class__.__name__}' has no attribute '{name}'")
        self._history.append(("get", name))
        return self._data[name]
    
    def __setattr__(self, name: str, value: Any) -> None:
        if name.startswith('_'):
            super().__setattr__(name, value)
            return
        
        print(f"Setting {name} = {value}")
        self._data[name] = value
        self._history.append(("set", name, value))
    
    def __delattr__(self, name: str) -> None:
        if name.startswith('_'):
            super().__delattr__(name)
            return
        
        print(f"Deleting {name}")
        if name not in self._data:
            raise AttributeError(f"'{self.__class__.__name__}' has no attribute '{name}'")
        del self._data[name]
        self._history.append(("del", name))
    
    def get_history(self) -> List[tuple]:
        return self._history

# Example usage
obj = AttributeTracker()
obj.name = "Alice"
obj.age = 30
print(obj.name)
del obj.age
print("History:", obj.get_history())

## 4. Custom Container Implementation 📦

In [ ]:
from typing import Any, Iterator, Optional

class CircularBuffer:
    def __init__(self, size: int):
        self.size = size
        self._buffer = [None] * size
        self._head = 0
        self._tail = 0
        self._count = 0
    
    def __len__(self) -> int:
        return self._count
    
    def __getitem__(self, index: int) -> Any:
        if not 0 <= index < self._count:
            raise IndexError("Buffer index out of range")
        return self._buffer[(self._head + index) % self.size]
    
    def __setitem__(self, index: int, value: Any) -> None:
        if not 0 <= index < self._count:
            raise IndexError("Buffer index out of range")
        self._buffer[(self._head + index) % self.size] = value
    
    def __iter__(self) -> Iterator:
        pos = self._head
        for _ in range(self._count):
            yield self._buffer[pos]
            pos = (pos + 1) % self.size
    
    def __contains__(self, item: Any) -> bool:
        return item in self._buffer[:self._count]
    
    def __str__(self) -> str:
        return f"CircularBuffer({list(self)})"
    
    def append(self, item: Any) -> None:
        self._buffer[self._tail] = item
        self._tail = (self._tail + 1) % self.size
        if self._count < self.size:
            self._count += 1
        else:
            self._head = (self._head + 1) % self.size

# Example usage
buffer = CircularBuffer(3)
for i in range(5):
    buffer.append(i)
    print(f"Buffer after adding {i}: {buffer}")

## 5. Advanced Descriptor Implementation 📝

In [ ]:
from typing import Any, Dict, Type
from datetime import datetime

class Validator:
    def __init__(self, validation_func, error_message: str):
        self.validation_func = validation_func
        self.error_message = error_message

class ValidatedField:
    def __init__(self, *validators: Validator):
        self.validators = validators
        self.name = None
    
    def __set_name__(self, owner: Type, name: str) -> None:
        self.name = name
    
    def __get__(self, instance: Any, owner: Type) -> Any:
        if instance is None:
            return self
        return instance.__dict__.get(self.name)
    
    def __set__(self, instance: Any, value: Any) -> None:
        for validator in self.validators:
            if not validator.validation_func(value):
                raise ValueError(
                    f"{self.name}: {validator.error_message}")
        instance.__dict__[self.name] = value

class AuditedField:
    def __init__(self):
        self.name = None
        self.audit_name = None
    
    def __set_name__(self, owner: Type, name: str) -> None:
        self.name = name
        self.audit_name = f"_{name}_audit"
    
    def __get__(self, instance: Any, owner: Type) -> Any:
        if instance is None:
            return self
        return instance.__dict__.get(self.name)
    
    def __set__(self, instance: Any, value: Any) -> None:
        old_value = instance.__dict__.get(self.name)
        instance.__dict__[self.name] = value
        
        if not hasattr(instance, self.audit_name):
            instance.__dict__[self.audit_name] = []
        
        audit_record = {
            'timestamp': datetime.now(),
            'old_value': old_value,
            'new_value': value
        }
        instance.__dict__[self.audit_name].append(audit_record)

# Example usage
class Person:
    name = ValidatedField(
        Validator(lambda x: isinstance(x, str), "Must be a string"),
        Validator(lambda x: len(x) >= 2, "Must be at least 2 characters")
    )
    age = ValidatedField(
        Validator(lambda x: isinstance(x, int), "Must be an integer"),
        Validator(lambda x: 0 <= x <= 150, "Must be between 0 and 150")
    )
    salary = AuditedField()

# Test the implementation
person = Person()
person.name = "Alice"
person.age = 30
person.salary = 50000
person.salary = 55000

print(f"Name: {person.name}")
print(f"Age: {person.age}")
print(f"Salary: {person.salary}")
print("Salary audit trail:")
for record in person._salary_audit:
    print(f"  {record['timestamp']}: {record['old_value']} -> {record['new_value']}")

## 6. Custom Context Manager 🔄

In [ ]:
from typing import Optional, List
from contextlib import contextmanager
import time

class ResourceManager:
    def __init__(self, name: str, timeout: float = 1.0):
        self.name = name
        self.timeout = timeout
        self.acquired = False
        self._start_time: Optional[float] = None
        self._operations: List[str] = []
    
    def __enter__(self) -> 'ResourceManager':
        print(f"Acquiring resource: {self.name}")
        self.acquired = True
        self._start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb) -> bool:
        duration = time.time() - self._start_time
        if duration > self.timeout:
            print(f"Warning: Resource {self.name} held for {duration:.2f}s")
        
        print(f"Releasing resource: {self.name}")
        print(f"Operations performed: {', '.join(self._operations)}")
        
        self.acquired = False
        return False  # Don't suppress exceptions
    
    def perform_operation(self, operation: str) -> None:
        if not self.acquired:
            raise RuntimeError("Resource not acquired")
        print(f"Performing: {operation}")
        self._operations.append(operation)

@contextmanager
def resource_group(*resources: ResourceManager):
    acquired = []
    try:
        for resource in resources:
            resource.__enter__()
            acquired.append(resource)
        yield resources
    finally:
        for resource in reversed(acquired):
            resource.__exit__(None, None, None)

# Example usage
db = ResourceManager("Database")
cache = ResourceManager("Cache")

# Single resource
with db as database:
    database.perform_operation("Query users")
    time.sleep(0.5)
    database.perform_operation("Update user")

# Multiple resources
with resource_group(db, cache) as resources:
    db, cache = resources
    db.perform_operation("Query data")
    cache.perform_operation("Cache result")
    time.sleep(0.5)

## 7. Advanced Comparison and Ordering 🔍

In [ ]:
from functools import total_ordering
from typing import Any, Optional
from datetime import datetime

@total_ordering
class SemanticVersion:
    def __init__(self, major: int, minor: int, patch: int,
                 pre_release: Optional[str] = None):
        self.major = major
        self.minor = minor
        self.patch = patch
        self.pre_release = pre_release
    
    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, SemanticVersion):
            return NotImplemented
        return (self.major == other.major and
                self.minor == other.minor and
                self.patch == other.patch and
                self.pre_release == other.pre_release)
    
    def __lt__(self, other: Any) -> bool:
        if not isinstance(other, SemanticVersion):
            return NotImplemented
        
        if self.major != other.major:
            return self.major < other.major
        if self.minor != other.minor:
            return self.minor < other.minor
        if self.patch != other.patch:
            return self.patch < other.patch
        
        # Pre-release versions are lower than release versions
        if self.pre_release is None and other.pre_release is not None:
            return False
        if self.pre_release is not None and other.pre_release is None:
            return True
        return (self.pre_release or '') < (other.pre_release or '')
    
    def __str__(self) -> str:
        version = f"{self.major}.{self.minor}.{self.patch}"
        if self.pre_release:
            version += f"-{self.pre_release}"
        return version
    
    def __repr__(self) -> str:
        return f"SemanticVersion({self.major}, {self.minor}, {self.patch}, {repr(self.pre_release)})"

# Example usage
versions = [
    SemanticVersion(1, 0, 0, "alpha"),
    SemanticVersion(1, 0, 0),
    SemanticVersion(1, 1, 0),
    SemanticVersion(2, 0, 0, "beta"),
    SemanticVersion(2, 0, 0)
]

print("Unsorted versions:")
for v in versions:
    print(f"  {v}")

print("\nSorted versions:")
for v in sorted(versions):
    print(f"  {v}")

## 8. Real-World Example: Database ORM 🗃️

In [ ]:
from typing import Any, Dict, List, Type, TypeVar, Optional
from datetime import datetime

T = TypeVar('T', bound='Model')

class Field:
    def __init__(self, field_type: Type, required: bool = True,
                 default: Any = None):
        self.field_type = field_type
        self.required = required
        self.default = default
        self.name = None
    
    def __set_name__(self, owner: Type, name: str) -> None:
        self.name = name
    
    def __get__(self, instance: Any, owner: Type) -> Any:
        if instance is None:
            return self
        return instance.__dict__.get(self.name, self.default)
    
    def __set__(self, instance: Any, value: Any) -> None:
        if value is None and self.required:
            raise ValueError(f"{self.name} is required")
        if value is not None and not isinstance(value, self.field_type):
            try:
                value = self.field_type(value)
            except:
                raise TypeError(
                    f"{self.name} must be of type {self.field_type.__name__}")
        instance.__dict__[self.name] = value

class ModelMeta(type):
    def __new__(mcs, name: str, bases: tuple, namespace: dict) -> Type:
        fields = {}
        for key, value in namespace.items():
            if isinstance(value, Field):
                fields[key] = value
        
        namespace['_fields'] = fields
        return super().__new__(mcs, name, bases, namespace)

class Model(metaclass=ModelMeta):
    def __init__(self, **kwargs):
        for name, field in self._fields.items():
            setattr(self, name, kwargs.get(name, field.default))
        
        self._modified = False
        self._created_at = datetime.now()
        self._updated_at = self._created_at
    
    def __setattr__(self, name: str, value: Any) -> None:
        if name in self._fields:
            self._modified = True
            self._updated_at = datetime.now()
        super().__setattr__(name, value)
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            name: getattr(self, name)
            for name in self._fields
        }
    
    @classmethod
    def from_dict(cls: Type[T], data: Dict[str, Any]) -> T:
        return cls(**data)
    
    def __str__(self) -> str:
        fields = [f"{name}={getattr(self, name)}"
                 for name in self._fields]
        return f"{self.__class__.__name__}({', '.join(fields)})"

# Example usage
class User(Model):
    id = Field(int, required=True)
    name = Field(str, required=True)
    email = Field(str, required=True)
    age = Field(int, required=False, default=0)
    active = Field(bool, required=False, default=True)

# Create a user
user = User(
    id=1,
    name="Alice",
    email="alice@example.com",
    age=30
)

print(f"User: {user}")
print(f"Dictionary: {user.to_dict()}")

# Modify user
user.age = 31
print(f"Modified: {user._modified}")
print(f"Updated at: {user._updated_at}")

# Create from dictionary
data = {
    'id': 2,
    'name': 'Bob',
    'email': 'bob@example.com'
}
user2 = User.from_dict(data)
print(f"User2: {user2}")

## 9. Practice Exercises 🎯

### Exercise 1: Implement a Custom Collection

In [ ]:
class UniqueList:
    """Implement a list that only stores unique elements and maintains insertion order.
    
    Required dunder methods:
    - __init__
    - __len__
    - __getitem__
    - __setitem__
    - __delitem__
    - __contains__
    - __iter__
    - __str__
    - __add__
    """
    pass

### Exercise 2: Create a Property Descriptor

In [ ]:
class TypedProperty:
    """Implement a property descriptor that enforces type checking and validation.
    
    Required dunder methods:
    - __init__
    - __set_name__
    - __get__
    - __set__
    """
    pass

### Exercise 3: Build a Context Manager Stack

In [ ]:
class ContextStack:
    """Implement a context manager that can manage multiple resources.
    
    Required dunder methods:
    - __init__
    - __enter__
    - __exit__
    - __len__
    - __str__
    """
    pass

## 10. Advanced Dunder Patterns 🚀

### 10.1 Custom Number System Implementation

In [ ]:
class ComplexNumber:
    def __init__(self, real: float, imag: float):
        self.real = real
        self.imag = imag
    
    def __add__(self, other: 'ComplexNumber') -> 'ComplexNumber':
        if isinstance(other, (int, float)):
            other = ComplexNumber(other, 0)
        return ComplexNumber(self.real + other.real,
                           self.imag + other.imag)
    
    def __sub__(self, other: 'ComplexNumber') -> 'ComplexNumber':
        if isinstance(other, (int, float)):
            other = ComplexNumber(other, 0)
        return ComplexNumber(self.real - other.real,
                           self.imag - other.imag)
    
    def __mul__(self, other: 'ComplexNumber') -> 'ComplexNumber':
        if isinstance(other, (int, float)):
            other = ComplexNumber(other, 0)
        return ComplexNumber(
            self.real * other.real - self.imag * other.imag,
            self.real * other.imag + self.imag * other.real
        )
    
    def __truediv__(self, other: 'ComplexNumber') -> 'ComplexNumber':
        if isinstance(other, (int, float)):
            other = ComplexNumber(other, 0)
        denominator = other.real**2 + other.imag**2
        return ComplexNumber(
            (self.real * other.real + self.imag * other.imag) / denominator,
            (self.imag * other.real - self.real * other.imag) / denominator
        )
    
    def __abs__(self) -> float:
        return (self.real**2 + self.imag**2) ** 0.5
    
    def __eq__(self, other: 'ComplexNumber') -> bool:
        if isinstance(other, (int, float)):
            other = ComplexNumber(other, 0)
        return self.real == other.real and self.imag == other.imag
    
    def __str__(self) -> str:
        if self.imag >= 0:
            return f"{self.real} + {self.imag}i"
        return f"{self.real} - {abs(self.imag)}i"
    
    def __repr__(self) -> str:
        return f"ComplexNumber({self.real}, {self.imag})"

# Example usage
c1 = ComplexNumber(2, 3)
c2 = ComplexNumber(1, -1)

print(f"c1 = {c1}")
print(f"c2 = {c2}")
print(f"c1 + c2 = {c1 + c2}")
print(f"c1 * c2 = {c1 * c2}")
print(f"|c1| = {abs(c1)}")

### 10.2 Advanced Iterator Implementation

In [ ]:
from typing import Iterator, Any, Optional

class PeekableIterator:
    def __init__(self, iterable):
        self._iterator = iter(iterable)
        self._peek_value: Optional[Any] = None
        self._has_peek = False
    
    def __iter__(self) -> 'PeekableIterator':
        return self
    
    def __next__(self) -> Any:
        if self._has_peek:
            value = self._peek_value
            self._has_peek = False
            self._peek_value = None
            return value
        return next(self._iterator)
    
    def peek(self) -> Any:
        if not self._has_peek:
            self._peek_value = next(self._iterator)
            self._has_peek = True
        return self._peek_value

class WindowIterator:
    def __init__(self, iterable, window_size: int):
        self._iterator = iter(iterable)
        self._window_size = window_size
        self._buffer = []
    
    def __iter__(self) -> 'WindowIterator':
        return self
    
    def __next__(self) -> list:
        while len(self._buffer) < self._window_size:
            try:
                self._buffer.append(next(self._iterator))
            except StopIteration:
                if not self._buffer:
                    raise
                break
        
        result = self._buffer[:]
        self._buffer = self._buffer[1:]
        return result

# Example usage
numbers = [1, 2, 3, 4, 5]

# Peekable Iterator
print("Peekable Iterator:")
pit = PeekableIterator(numbers)
print(f"Peek: {pit.peek()}")
print(f"Next: {next(pit)}")
print(f"Next: {next(pit)}")

# Window Iterator
print("\nWindow Iterator (size 3):")
wit = WindowIterator(numbers, 3)
for window in wit:
    print(f"Window: {window}")

### 10.3 Custom Container with Slicing

In [ ]:
from typing import Any, List, Union, Tuple

class DataSeries:
    def __init__(self, data: List[Any], name: str = ""):
        self._data = data
        self.name = name
    
    def __len__(self) -> int:
        return len(self._data)
    
    def __getitem__(self, key: Union[int, slice]) -> Union['DataSeries', Any]:
        if isinstance(key, slice):
            return DataSeries(self._data[key], self.name)
        return self._data[key]
    
    def __setitem__(self, key: Union[int, slice], value: Any) -> None:
        self._data[key] = value
    
    def __iter__(self) -> Iterator:
        return iter(self._data)
    
    def __reversed__(self) -> Iterator:
        return reversed(self._data)
    
    def __add__(self, other: Union['DataSeries', List, int, float]) -> 'DataSeries':
        if isinstance(other, (int, float)):
            return DataSeries([x + other for x in self._data], self.name)
        other_data = other._data if isinstance(other, DataSeries) else other
        if len(self) != len(other_data):
            raise ValueError("Length mismatch")
        return DataSeries([a + b for a, b in zip(self._data, other_data)],
                         self.name)
    
    def __mul__(self, other: Union['DataSeries', List, int, float]) -> 'DataSeries':
        if isinstance(other, (int, float)):
            return DataSeries([x * other for x in self._data], self.name)
        other_data = other._data if isinstance(other, DataSeries) else other
        if len(self) != len(other_data):
            raise ValueError("Length mismatch")
        return DataSeries([a * b for a, b in zip(self._data, other_data)],
                         self.name)
    
    def __str__(self) -> str:
        name = f" '{self.name}'" if self.name else ""
        return f"DataSeries{name}[{', '.join(map(str, self._data))}]"
    
    def apply(self, func) -> 'DataSeries':
        return DataSeries([func(x) for x in self._data], self.name)
    
    def filter(self, predicate) -> 'DataSeries':
        return DataSeries([x for x in self._data if predicate(x)], self.name)

# Example usage
data1 = DataSeries([1, 2, 3, 4, 5], "series1")
data2 = DataSeries([10, 20, 30, 40, 50], "series2")

print(f"data1: {data1}")
print(f"data2: {data2}")
print(f"data1 + data2: {data1 + data2}")
print(f"data1 * 2: {data1 * 2}")
print(f"First 3 elements: {data1[:3]}")
print(f"Squared values: {data1.apply(lambda x: x**2)}")
print(f"Even numbers: {data1.filter(lambda x: x % 2 == 0)}")

## 11. Real-World Application: Event System 🎮

In [ ]:
from typing import Dict, List, Callable, Any
from datetime import datetime
from uuid import uuid4

class Event:
    def __init__(self, event_type: str, data: Any = None):
        self.id = str(uuid4())
        self.type = event_type
        self.data = data
        self.timestamp = datetime.now()
    
    def __str__(self) -> str:
        return f"Event({self.type}, {self.data})"

class EventEmitter:
    def __init__(self):
        self._listeners: Dict[str, List[Callable]] = {}
        self._event_history: List[Event] = []
    
    def __iadd__(self, subscriber: tuple) -> 'EventEmitter':
        """Subscribe to events using += operator"""
        event_type, callback = subscriber
        self.on(event_type, callback)
        return self
    
    def __isub__(self, subscriber: tuple) -> 'EventEmitter':
        """Unsubscribe from events using -= operator"""
        event_type, callback = subscriber
        self.off(event_type, callback)
        return self
    
    def __call__(self, event_type: str, data: Any = None) -> None:
        """Emit event using function call syntax"""
        self.emit(event_type, data)
    
    def __len__(self) -> int:
        """Get number of event listeners"""
        return sum(len(callbacks) for callbacks in self._listeners.values())
    
    def on(self, event_type: str, callback: Callable) -> None:
        if event_type not in self._listeners:
            self._listeners[event_type] = []
        self._listeners[event_type].append(callback)
    
    def off(self, event_type: str, callback: Callable) -> None:
        if event_type in self._listeners:
            self._listeners[event_type].remove(callback)
    
    def emit(self, event_type: str, data: Any = None) -> None:
        event = Event(event_type, data)
        self._event_history.append(event)
        
        if event_type in self._listeners:
            for callback in self._listeners[event_type]:
                try:
                    callback(event)
                except Exception as e:
                    print(f"Error in event handler: {e}")
    
    def get_history(self, event_type: str = None) -> List[Event]:
        if event_type is None:
            return self._event_history
        return [e for e in self._event_history if e.type == event_type]

# Example usage
def on_user_login(event: Event):
    print(f"User logged in: {event.data}")

def on_user_logout(event: Event):
    print(f"User logged out: {event.data}")

# Create event emitter
events = EventEmitter()

# Subscribe to events using operators
events += ("login", on_user_login)
events += ("logout", on_user_logout)

# Emit events using function call syntax
events("login", {"user_id": 123, "username": "alice"})
events("logout", {"user_id": 123})

# Unsubscribe using operator
events -= ("login", on_user_login)

# Check history
print("\nEvent History:")
for event in events.get_history():
    print(f"{event.timestamp}: {event}")

## 12. Advanced Exercise: Build a Query Builder 🔍

In [ ]:
class QueryBuilder:
    """Implement a fluent SQL query builder with dunder methods.
    
    Required features:
    - Method chaining
    - Operator overloading for conditions
    - String representation
    - Validation
    """
    def __init__(self):
        self._select = []
        self._from = None
        self._where = []
        self._order_by = []
        self._group_by = []
        self._limit = None
    
    def __str__(self) -> str:
        """Convert query to SQL string"""
        parts = []
        
        # SELECT
        select_cols = ", ".join(self._select) if self._select else "*"
        parts.append(f"SELECT {select_cols}")
        
        # FROM
        if self._from:
            parts.append(f"FROM {self._from}")
        
        # WHERE
        if self._where:
            parts.append(f"WHERE {' AND '.join(self._where)}")
        
        # GROUP BY
        if self._group_by:
            parts.append(f"GROUP BY {', '.join(self._group_by)}")
        
        # ORDER BY
        if self._order_by:
            parts.append(f"ORDER BY {', '.join(self._order_by)}")
        
        # LIMIT
        if self._limit is not None:
            parts.append(f"LIMIT {self._limit}")
        
        return " ".join(parts)
    
    def __call__(self) -> str:
        """Execute query by returning SQL string"""
        return str(self)
    
    def select(self, *columns) -> 'QueryBuilder':
        self._select.extend(columns)
        return self
    
    def from_(self, table: str) -> 'QueryBuilder':
        self._from = table
        return self
    
    def where(self, condition: str) -> 'QueryBuilder':
        self._where.append(condition)
        return self
    
    def group_by(self, *columns) -> 'QueryBuilder':
        self._group_by.extend(columns)
        return self
    
    def order_by(self, column: str, desc: bool = False) -> 'QueryBuilder':
        direction = "DESC" if desc else "ASC"
        self._order_by.append(f"{column} {direction}")
        return self
    
    def limit(self, n: int) -> 'QueryBuilder':
        self._limit = n
        return self

# Example usage
query = QueryBuilder()
query.select("name", "age") \
     .from_("users") \
     .where("age >= 18") \
     .where("active = true") \
     .order_by("name") \
     .limit(10)

print("Generated SQL:")
print(query())

## 13. Best Practices for Dunder Methods 📚

1. Always return NotImplemented for unsupported operations
2. Implement related methods together (e.g., __eq__ with __hash__)
3. Use appropriate type hints
4. Document behavior clearly
5. Handle edge cases
6. Keep methods focused and simple
7. Follow Python's data model conventions

## 14. Common Pitfalls to Avoid ⚠️

1. Modifying __init__ arguments without copying
2. Forgetting to return self in chainable methods
3. Incorrect implementation of comparison methods
4. Inconsistent behavior between related methods
5. Not handling type errors properly
6. Recursive calls in __getattr__
7. Mutable default arguments

## 15. Summary 🎉

- Dunder methods provide powerful customization
- They enable Pythonic object behavior
- They support operator overloading
- They allow custom attribute access
- They enable context management

Keep exploring and experimenting with dunder methods to create more elegant and powerful Python code!
